# Reports

In [3]:
import os
import re

import numpy as np
import pandas as pd

# for the notebook rendering 
from IPython.display import display, HTML, FileLink, Markdown
from IPython.display import FileLink as FL
from IPython.display import Markdown as MD


DIR = os.getcwd()
print(f'{DIR=}')

RDIR = os.path.join(DIR,'reports')
print(f'{RDIR=}')

DIR='c:\\Users\\JGarza\\GitHub\\wifi_monitor'
RDIR='c:\\Users\\JGarza\\GitHub\\wifi_monitor\\reports'


In [4]:
data = {}

for i in os.listdir(RDIR):
    # print(i)
    try:

        text = ''
        with open(os.path.join(RDIR, i), 'r') as f:
            text = f.read()
        # print(text)

        wifi = re.findall('^SSID:.*\n',text)[0]
        wifi = wifi.replace('\n', '').replace('SSID: ','')

        pings = re.findall('.*time=.*\n',text)
        pings = [ re.sub('.*time=','',p) for p in pings]
        pings = [ re.sub('ms.*\n','',p) for p in pings]
        pings = [ int(p) for p in pings]
        # print(*pings,sep='\n')

        timeouts = re.findall('Request timed out.\n',text)

        trycount = re.findall('(Request|Reply).*\n',text)

        if wifi in data.keys():
            pass
        else:
            data[wifi] = {}
            data[wifi]['pings'] = []
            data[wifi]['timeouts'] = []
            data[wifi]['trycount'] = []
        
        data[wifi]['pings'].extend(pings)
        data[wifi]['timeouts'].extend([len(timeouts)])
        data[wifi]['trycount'].extend([len(trycount)])

    except Exception as e:
        pass
        # print(i,e)

for k in data.keys():
    print('-'*20)
    print(k)
    df_pings = pd.DataFrame(data[k]['pings'])
    print(df_pings[0].describe())
    print()
    print(f'Request timed out:')
    print(f'{ sum(data[k]['timeouts']) } out of { sum(data[k]['trycount']) }')
    print(f'{ sum(data[k]['timeouts']) / sum(data[k]['trycount']) }')
    # print(f'"Request timed out" : { sum(data[k]['timeouts']) } out of {}')
    # df_to = pd.DataFrame(data[k]['timeouts'])
    # print(df_to[0].describe())
    # print(df_to)


--------------------
Error 404 NH
count    500.000000
mean      19.094000
std       26.866872
min        9.000000
25%       11.000000
50%       12.000000
75%       15.000000
max      279.000000
Name: 0, dtype: float64

Request timed out:
0 out of 500
0.0
--------------------
Error 404
count    499.000000
mean      11.430862
std        5.030502
min        7.000000
25%        9.000000
50%       10.000000
75%       12.000000
max       66.000000
Name: 0, dtype: float64

Request timed out:
1 out of 500
0.002
--------------------
Error 404_EXT
count    499.000000
mean      15.422846
std        5.193017
min        9.000000
25%       13.000000
50%       14.000000
75%       16.000000
max       55.000000
Name: 0, dtype: float64

Request timed out:
1 out of 500
0.002
